In [1]:
import pandas as pd
import psycopg2.extras
conn = psycopg2.connect("dbname='test' user='test' host='/tmp/'")
cur = conn.cursor()

En **SQL** por lo general trabajamos con una gran cantidad de tablas. Muchas de ellas tienen atributos en común, pero con diferente información en cada una de ellas. Observemos el siguiente ejemplo:

—_En las siguientes tablas (`cities` y `countries`) encontramos características socioeconómicas y características geográficas respectivamente. El objetivo de este ejercicio es; "crear una consulta que solo contenga tres atributos, `name` de la tabla `cities`, `name` de la tabla `countries` y `region` de la tabla `countries`". Ambas tablas contienen un atributo en común, es la misma información pero con un nombre distinto. Nos referimos a `code` en `countries` y `cities` en `conutry_code`. A estos casos los nombramos como `key_values` y con el realizamos el "match" de los registros entre ambas tablas._ — Al final les darémos un "alias" a cada uno de ellos.

En esta ocasión veremos que dentro de la sintaxis del _query_ aparecerán unos guiones de la siguiente forma `--`, seguidos de un texto. Como buenas prácticas en **SQL** esto quiere decir que podemos comentar esa parte del código y de esta manera saber ¿qué es lo que está haciendo cada una de las líneas antes de ser ejecutadas?...

Así, si alguien más utiliza nuestro código puede ver que es lo que intentamos hacer.

In [4]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    --4. Seleccionamos los nombres de los atributos (con un alias) de ser el caso.
    SELECT cities.name AS city, countries.name AS country, region
        
        --3. La tabla, con los atributos, que deseamos tener "a la izquierda de la consulta".
        FROM joindata_sql.cities 
        
        --2.- Invocación de INNER para "fijar a la derecha los atributos de la consulta". 
        INNER JOIN joindata_sql.countries
            
            --1.- Elección del atributo (key), de ambas tablas, con el cuál deseamos coincidir.
            ON cities.country_code =countries.code;
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,city,country,region
0,Abidjan,Cote d'Ivoire,Western Africa
1,Abu Dhabi,United Arab Emirates,Middle East
2,Abuja,Nigeria,Western Africa
3,Accra,Ghana,Western Africa
4,Addis Ababa,Ethiopia,Eastern Africa
5,Ahmedabad,India,Southern and Central Asia
6,Alexandria,Egypt,Northern Africa
7,Algiers,Algeria,Northern Africa
8,Almaty,Kazakhstan,Southern and Central Asia
9,Ankara,Turkey,Middle East


En ocasiones es mejor **"no"** escribir el nombre completo de las tablas como parte de la línea `SELECT ...` de nuestra sintaxis. Podemos poner alias en esta parte de nuestro statement para referirnos después, dentro de la línea `FROM`, a las tablas y a los esquemas en nuestras consultas.

—_El siguiente query obtiene, de la tabla `economies`,  los atributos `name`, `year` y `inflation_rate`. Asociando el atributo `code`, de la tabla `countries`, con el atributo `code` de la tabla `economies`:_ —

In [3]:
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    -- 3. Construcción del alias ("c") de la tabla "countries" y elección de los atributos (name, year, etc, etc...) de la tabla "economies" como 
    --    parte de nuestro query.
    SELECT c.code AS country_code, name, year, inflation_rate
    FROM joindata_sql.countries AS c
    
      -- 1. Countrucción del alias ("e") de la tabla "economies" para ser el objetivo de nuestro JOIN.
      INNER JOIN joindata_sql.economies AS e
      
        -- 2. Selección del atributo "code", de ambas tablas, como "key_value" para realizar el JOIN. Hay que tomar los alias de las tablas country 
        --    y economies, respectivamente, para llevarlo a cabo.
        ON c.code = e.code;
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,country_code,name,year,inflation_rate
0,AFG,Afghanistan,2010,2.179
1,AFG,Afghanistan,2015,-1.549
2,AGO,Angola,2010,14.480
3,AGO,Angola,2015,10.287
4,ALB,Albania,2010,3.605
5,ALB,Albania,2015,1.896
6,ARE,United Arab Emirates,2010,0.878
7,ARE,United Arab Emirates,2015,4.070
8,ARG,Argentina,2010,10.461
9,ARG,Argentina,2015,NaN
